In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import auto_vec_filter_helper.auto_vec_filter_data as avf_data
import seaborn as sns
from matplotlib.colors import ListedColormap
import math


In [ ]:
df = pd.read_csv("aggregate/results.tsv", sep='\t')
df_mean = df.groupby(["device","arch-flags","function","variant","SIMD-elementCount", "byte_size","element_size"])["time_ns"].mean().reset_index()
df_mean["tput GiB/s"] = (df_mean["byte_size"]/1024/1024/1024)/(df_mean["time_ns"]*1e-9)
df_mean["beauty_bytes"] = df_mean["byte_size"].apply(lambda x: avf_data.beautify_data_size(x))
df_mean["beauty_bytes_no_ending"] = df_mean["byte_size"].apply(lambda x: avf_data.beautify_data_size(x, False))
df_mean = df_mean[ (df_mean["arch-flags"] == "FPGA") ]
max_val = df_mean["tput GiB/s"].max()
print( max_val )
for es in [4,8]:
    df_mean[ df_mean["element_size"] == es ].plot(kind="bar", x="beauty_bytes", y="tput GiB/s", xlabel="Byte Size", ylabel="Throughput [GiB/s]",legend=False, title=f"Element Size in Bytes: {es}", ylim=(0,max_val))


In [ ]:
df_mean[ df_mean["element_size"] == 8 ].plot(kind="bar", x="beauty_bytes_no_ending", y="tput GiB/s", xlabel="Processed Data Size", ylabel="Throughput [GiB/s]",legend=False, ylim=(0,3))

In [ ]:

sns.set(style="whitegrid", rc={'grid.linestyle': '--'})
g = sns.barplot(data=df_mean[ df_mean["element_size"] == 8 ],  x="beauty_bytes_no_ending", y="tput GiB/s", hue="SIMD-elementCount", palette=sns.color_palette("muted"))
#g.set(xlabel="Processed Data Size [MiB]", ylabel="Throughput [GiB/s]")
g.set_ylim(0,3)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlabel("Processed Data Size [MiB]", fontsize=22)
plt.ylabel("Throughput [GiB/s]", fontsize=22)
plt.legend().remove()
plt.tight_layout()
plt.savefig("aggregate/aggregate_throughput.pdf")



In [ ]:
df = pd.read_csv("aggregate/aggregate_autovec_results.tsv", sep='\t')
df_mean = df.groupby(["device","compiler","version","arch-flags","function","variant","SIMD-elementCount","byte_size","element_size","element_count"])["time_ns"].mean().reset_index()
df_mean["tput GiB/s"] = (df_mean["byte_size"]/1024/1024/1024)/(df_mean["time_ns"]*1e-9)
df_mean["beauty_bytes"] = df_mean["byte_size"].apply(lambda x: avf_data.beautify_data_size(x))
df_mean["beauty_bytes_no_ending"] = df_mean["byte_size"].apply(lambda x: avf_data.beautify_data_size(x, False))
df_mean = df_mean[ (df_mean["arch-flags"] == "FPGA") ]
df_mean["color_id"]=df_mean["SIMD-elementCount"].apply(lambda x: int(math.log2(x)))
df_mean["SIMD-elementCount"]=df_mean["SIMD-elementCount"].apply(lambda x: int(x))

In [ ]:
sns.set(style="whitegrid", rc={'grid.linestyle': '--'})
cm = sns.color_palette('coolwarm', as_cmap=True)
g = sns.barplot(data=df_mean,  x="beauty_bytes_no_ending", y="tput GiB/s", hue="SIMD-elementCount", palette=sns.color_palette("muted"))
#set legend below the plot
g.legend(title="VL", loc='upper center', bbox_to_anchor=(0.5, -0.3), ncol=4)
#add shadow to the legend
plt.setp(g.get_legend().get_texts(), fontsize='18') 
plt.setp(g.get_legend().get_title(), fontsize='20')
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlabel("Processed Data Size [MiB]", fontsize=22)
plt.ylabel("Throughput [GiB/s]", fontsize=22)
plt.tight_layout()
plt.savefig("aggregate/aggregate_autovec_all_throughput.pdf")

In [ ]:
sns.set(style="whitegrid", rc={'grid.linestyle': '--'})
cm = sns.color_palette('coolwarm', as_cmap=True)
g = sns.barplot(data=df_mean[df_mean["SIMD-elementCount"]<16],  x="beauty_bytes_no_ending", y="tput GiB/s", hue="SIMD-elementCount", palette=sns.color_palette("muted"))
g.legend(title="VL", loc='upper center', bbox_to_anchor=(0.5, -0.3), ncol=4)

plt.setp(g.get_legend().get_texts(), fontsize='18') 
plt.setp(g.get_legend().get_title(), fontsize='20')
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlabel("Processed Data Size [MiB]", fontsize=22)
plt.ylabel("Throughput [GiB/s]", fontsize=22)
plt.tight_layout()
plt.savefig("aggregate/aggregate_autovec_128_to_256_throughput.pdf")